全局函数

In [1]:
from pprint import pprint

def to_time(value, ratio = 1):
    return 60 / value * ratio

to_bpm = to_time

类

In [2]:
class SonolusServer():
    pass

客户端

In [3]:
# import urllib
# import urllib.request
import urllib.error
import urllib.parse

# pprint(urllib.parse.urlparse('https://servers.sonolus.com/pjsekai/sonolus/info?localization=zhs'))


服务端

In [4]:
import http
import http.server
import http.client
import gzip
import json

class toolbox_handler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        # pprint(self.path)
        # pprint(self.headers)
        # pprint(self.command)
        self.headers.replace_header('Host', HOST)
        
        # https://servers.sonolus.com/pjsekai/sonolus/levels/pjsekai-315-782-master/data?1.1.0
        # /pjsekai/sonolus/levels/pjsekai-315-782-master/data?1.1.0
        
        connection = http.client.HTTPSConnection(host = HOST)
        connection.request(method = self.command, url = self.path, headers = self.headers)
        response = connection.getresponse()

        self.send_response(response.status)

        for k, v in response.getheaders():
            if k == 'Sonolus-Version':
                self.send_header(k, v)
            
            if k == 'Content-Type':
                self.send_header(k, v)
                # if 'json' in v:
                #     is_json = True
            
            if k == 'Content-Encoding':
                self.send_header(k, v)
                # if v == 'gzip':
                #     is_gzip = True
        
        self.end_headers()
        
        if self.path == '/pjsekai/sonolus/levels/pjsekai-315-782-master/data?1.1.0':
            content = json.loads(gzip.decompress(response.read()).decode())
            content['entities'].sort(key = lambda e:e['data'][0]['value'] if e['data'] != [] and e['data'][0]['name'] == '#BEAT' else -1)
            self.wfile.write(json.dumps(content).encode())

        # pprint(gzip.decompress(response.read()).decode())

        else:
            self.wfile.write(response.read())
        connection.close()

    def do_POST(self):
        # pprint(self.path)
        # pprint(self.headers)
        # pprint(self.command)
        pass

主函数

In [5]:
PORT = 9639
HOST = 'servers.sonolus.com'

toolbox_server = http.server.ThreadingHTTPServer(('', PORT), toolbox_handler)

toolbox_server.serve_forever()

127.0.0.1 - - [29/Jun/2023 09:52:56] "GET /pjsekai/sonolus/levels/pjsekai-315-782-master/data?1.1.0 HTTP/1.1" 200 -


KeyboardInterrupt: 

测试

In [ ]:
import unittest